# Model recovery attack: `wine`

In [2]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import mra
from mra import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

xy = np.loadtxt("../datasets/wine-white.csv", delimiter=";")
np.random.shuffle(xy)
x = xy[:,0:-1]
y = xy[:,-1]-3

min_values = np.array([min(x[:,i]) for i in range(x.shape[1])])
max_values = np.array([max(x[:,i]) for i in range(x.shape[1])])
x = (x-min_values)/(max_values - min_values)
y = y.reshape((len(x), 1)).astype("float32")
train_ds = make_dataset(x, y, lambda t: t)

2022-02-17 14:54:52.272567: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-17 14:54:52.272634: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: xgpg0
2022-02-17 14:54:52.272666: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: xgpg0
2022-02-17 14:54:52.272803: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 495.29.5
2022-02-17 14:54:52.272837: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 495.29.5
2022-02-17 14:54:52.272846: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 495.29.5
2022-02-17 14:54:52.273420: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perfor

In [4]:
mra_wine_4 = mra(train_ds)

In [5]:
def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    output = tf.keras.layers.Dense(64, activation="relu")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    # x = tf.keras.layers.Dense(64, activation="relu")(xin)
    # x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(7, activation="softmax")(xin)
    return tf.keras.Model(xin, output)

In [7]:
log_train = mra_wine_4.train(
    make_f,
    make_g,
    loss_fn=tf.keras.losses.sparse_categorical_crossentropy,
    batch_size=32,
    epoch=5,
    lr=0.001,
    verbose=True, log_every=50
    )

Iteration 0001: Training loss: 1.9355
Iteration 0051: Training loss: 1.1697
Iteration 0101: Training loss: 1.4907
Iteration 0151: Training loss: 1.1881
Iteration 0201: Training loss: 1.0032
Iteration 0251: Training loss: 0.9215
Iteration 0301: Training loss: 0.9813
Iteration 0351: Training loss: 1.2622
Iteration 0401: Training loss: 1.1110
Iteration 0451: Training loss: 0.9986
Iteration 0501: Training loss: 1.1118
Iteration 0551: Training loss: 1.1252
Iteration 0601: Training loss: 1.2032
Iteration 0651: Training loss: 1.1416
Iteration 0701: Training loss: 0.9887
Iteration 0751: Training loss: 0.9387


In [8]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(512, activation=act)(xin)
    x = tf.keras.layers.Dense(256, activation=act)(x)
    x = tf.keras.layers.Dense(128, activation=act)(x)
    x = tf.keras.layers.Dense(64, activation=act)(x)
    x = tf.keras.layers.Dense(32, activation=act)(x)
    x = tf.keras.layers.Dense(11, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [9]:
log_attack = mra_wine_4.attack(
    attack_iter=100,
    make_generator=make_generator,
    input_noise="normal",
    input_z=True,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.1615, RG-normal: 0.1641, reconstruction validation: 0.0473
Iteration 0002: RG-uniform: 0.1832, RG-normal: 0.1492, reconstruction validation: 0.0233
Iteration 0003: RG-uniform: 0.1703, RG-normal: 0.1620, reconstruction validation: 0.0180
Iteration 0004: RG-uniform: 0.1819, RG-normal: 0.1622, reconstruction validation: 0.0121
Iteration 0005: RG-uniform: 0.1786, RG-normal: 0.1618, reconstruction validation: 0.0138
Iteration 0006: RG-uniform: 0.1871, RG-normal: 0.1602, reconstruction validation: 0.0092
Iteration 0007: RG-uniform: 0.1976, RG-normal: 0.1716, reconstruction validation: 0.0100
Iteration 0008: RG-uniform: 0.1852, RG-normal: 0.1589, reconstruction validation: 0.0079
Iteration 0009: RG-uniform: 0.1921, RG-normal: 0.1394, reconstruction validation: 0.0092
Iteration 0010: RG-uniform: 0.1503, RG-normal: 0.1614, reconstruction validation: 0.0090
Iteration 0011: RG-uniform: 0.1700, RG-normal: 0.1589, reconstruction validation: 0.0112
Iteration 0012: RG-un

In [10]:
np.savetxt("./log/wine-white-4.csv", log_attack, delimiter=",")